# task no. 4

This is the notebook taking care of the task no.4 : feature prediction.

In our analysis we have decided very different models to see how they differ from eachother in term of performance and also their explainability to see if they make sense.

For this notebook we decided to explore the nature of NeuralNetworks on the dataset.

First a preparation is due to be used.

## dataset preparation



In [ ]:
import pandas as pd
import os


RACES_PATH=os.path.join("..","dataset","engineered_races.csv")

races_df=pd.read_csv(RACES_PATH)

First we binarize the columns

In [16]:
from sklearn.model_selection import train_test_split

races_df['position']=(races_df['position']>20).astype(int)
races_df['date']=pd.to_datetime(races_df['date'])


split_idx= races_df['date'].dt.year <= 2022

X_test_set=races_df.loc[~split_idx].drop(columns='position')

X_dev_set=races_df[split_idx].drop(columns='position')
Y_dev_set=races_df.loc[split_idx,'position']

X_train_set,X_val_set,Y_train_set,Y_val_set=train_test_split(
    X_dev_set,
    Y_dev_set,
    test_size=0.2,
    stratify=Y_dev_set,
    random_state=42
)



first a stratification can only help the generization capabilities.

Now we have to setup the task, for this kind of setting the binary cross entropy is the most appropriate given we just want to classify stuff and we are not doing any regression whatsoever.

A first test using a simple NN might be usefull in this case to see the most basic algorithm.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, initializers

initializer=initializers.HeNormal()

input_size=train_set.shape[1]

output_size=1

# Define the model
model = models.Sequential([
    layers.Dense(64, activation='relu', 
                 kernel_initializer=initializer,  # He initialization
                 bias_initializer=initializer,       # Bias initialized to zeros
                 input_shape=(input_size,)),                          # Input shape
    layers.Dense(32, activation='relu',
                 kernel_initializer=initializer),  # Xavier initialization
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy','f1-score']
)

model.fit(
    X_train_set,Y_train_set,
    epochs=100
    )